In [1]:
# Path setup
import sys
import os

sys.path.append("/home/dchen/Random_Forest_Weights/")

# Basics:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Helpful:
from sklearn.model_selection import train_test_split

# Pipeline and ColumnsTransformer:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
# models:
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor

# my functions:
from src_rf.methods.calc_mean import *
from src_rf.methods.calc_weights import *
from src_rf.methods.calc_dist import *

In [10]:
# Load cv results:
rf_cv_results = pd.read_csv("/Data/Delong_BA_Data/rf_weights/rf_cv_results.csv")
# Load cv 2 results:
rf_cv_results_2 = pd.read_csv("/Data/Delong_BA_Data/rf_weights/rf_cv_results_2.csv")
rf_cv_results_1_qrf_2 = pd.read_csv("/Data/Delong_BA_Data/rf_weights/qrf_2/rf_cv_results_1.csv")
rf_cv_results_1_qrf_3 = pd.read_csv("/Data/Delong_BA_Data/rf_weights/qrf_3/rf_cv_results_1.csv")

In [11]:
rf_cv_results.sort_values(['mean_test_score'], ascending = False)[['mean_test_score', 'std_test_score'
                                                                   , 'param_n_estimators', 'param_min_samples_split'
                                                                   ,'param_min_samples_leaf', 'param_max_depth'
                                                                   , 'param_bootstrap']].head(7)

,mean_test_score,std_test_score,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_bootstrap
189,-13.448995,3.019934,800,2,10,NaN,True
89,-13.453130,3.014812,700,5,10,50.0,True
169,-13.478841,3.028890,400,2,10,25.0,True
130,-13.501498,3.039728,200,2,10,25.0,True
58,-13.513956,2.887897,900,5,15,25.0,True
10,-13.522711,2.876794,600,5,15,NaN,True
174,-13.522711,2.876794,600,10,15,NaN,True


In [12]:
rf_cv_results_2.sort_values(['mean_test_score'], ascending = False)[['mean_test_score', 'std_test_score','param_max_samples'
                                                                   , 'param_n_estimators', 'param_min_samples_split'
                                                                   ,'param_min_samples_leaf', 'param_max_depth'
                                                                   , 'param_bootstrap']].head(15)

,mean_test_score,std_test_score,param_max_samples,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_bootstrap
76,-12.982787,2.979846,0.5,700,5,5,40.0,True
42,-12.994283,2.970620,0.5,600,2,5,60.0,True
7,-13.093425,2.847754,0.5,700,5,10,NaN,True
47,-13.103472,2.844889,0.5,600,7,10,NaN,True
50,-13.130063,3.028278,0.6,800,7,5,NaN,True
6,-13.138624,3.028270,0.6,700,5,5,NaN,True
59,-13.188964,2.897788,0.6,800,10,10,60.0,True
46,-13.198894,2.895032,0.6,700,10,10,50.0,True
55,-13.198894,2.895032,0.6,700,2,10,50.0,True
56,-13.198894,2.895032,0.6,700,3,10,50.0,True


In [13]:
rf_cv_results_1_qrf_2.sort_values(['mean_test_score'], ascending = False)[['mean_test_score', 'std_test_score','param_max_samples'
                                                                   , 'param_n_estimators', 'param_min_samples_split'
                                                                   ,'param_min_samples_leaf', 'param_max_depth'
                                                                   , 'param_bootstrap']].head(15)

,mean_test_score,std_test_score,param_max_samples,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_bootstrap
80,-25.620655,10.394204,0.3,700,7,5,50.0,True
87,-25.900708,10.303792,0.5,500,10,10,NaN,True
15,-25.900708,10.303792,0.5,500,7,10,NaN,True
53,-25.918492,10.306338,0.5,700,2,10,NaN,True
51,-25.918492,10.306338,0.5,700,10,10,50.0,True
31,-25.940437,10.339437,0.5,300,10,10,30.0,True
64,-26.030250,10.434046,0.5,100,7,10,NaN,True
5,-26.030250,10.434046,0.5,100,2,10,50.0,True
86,-26.353858,10.201762,0.7,100,2,10,50.0,True
83,-26.359197,10.216645,0.7,300,2,10,NaN,True


In [14]:
rf_cv_results_1_qrf_3.sort_values(['mean_test_score'], ascending = False)[['mean_test_score', 'std_test_score','param_max_samples'
                                                                   , 'param_n_estimators', 'param_min_samples_split'
                                                                   ,'param_min_samples_leaf', 'param_max_depth'
                                                                   , 'param_bootstrap']].head(15)

,mean_test_score,std_test_score,param_max_samples,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_bootstrap
91,-22.435893,2.799340,0.3,300,10,10,5.0,True
3,-22.446348,2.896342,0.3,700,5,5,5.0,True
67,-22.508754,2.886647,0.3,500,5,5,5.0,True
30,-22.755919,3.045708,0.5,500,5,5,5.0,True
8,-22.766634,3.004994,0.5,700,7,10,5.0,True
19,-22.772680,2.966227,0.5,700,5,20,5.0,True
36,-22.772680,2.966227,0.5,700,7,20,5.0,True
75,-22.798009,2.972031,0.5,500,10,20,5.0,True
70,-22.819040,2.943495,0.5,300,7,20,5.0,True
52,-22.905803,3.229924,0.7,700,10,10,5.0,True
